In [123]:
import numpy as np

In [124]:
def sigmoid(z):
    return 1./(1+np.exp(-z))

In [125]:
def log_loss(t, y):
    return np.sum((-t*np.log(y) - (1 - t) * (np.log(1 - y))), keepdims=True).flatten()

def diff_log_loss(t, y):
    return np.sum((t-y)*x, keepdims=True)

In [ ]:
def logos(sample):
    sigmoid(sample.dot(w) + b)

In [319]:
class Layer: #слой реализован в виде класса
  
    
    def __init__(self, n_inp, n_out, lr=0.1):
        self.shape = (n_inp, n_out) #залетают входящие и выходящие значения
        self.lr = lr #шаг обучения
        self.w = np.zeros(self.shape, dtype=np.float32) #веса в начале заданы нулями
        self.b = np.zeros((1, n_out), dtype=np.float32) #смещения в начале заданы нулями
        self._clear_grads() #проход обычного градиентного спуска

    def _clear_grads(self): #градиентный спуск
        self.inp = None
        self.activations = None
        self.d_sigma = None
        self.d_w = None
        self.d_b = None

    def __call__(self, x): #вызов класса
        if len(x.shape) == 1: #если входящие значения одноразмерны - сменить размерность
            x = x.reshape(1, -1)
        self.inp = x #задаем входные значения
        self.activations = sigmoid(x.dot(self.w) + self.b) #запуск функции с функцией активации
        return self.activations

    def backward(self, grad): # обратный проход
        self.d_sigma = self.activations * (1 - self.activations) #производная функции ошибки 
        self.d_w = self.grad_w(grad)
        self.d_b = self.grad_b(grad)
        return self.grad_x(grad)

    def grad_w(self, grad):
        return grad * self.inp.T * self.d_sigma

    def grad_b(self, grad):
        return grad * self.d_sigma

    def grad_x(self, grad):
        return self.w.dot(grad) * self.d_sigma

    def step(self):
        self.w -= self.d_w * self.lr
        self.b -= self.d_b * self.lr
        self._clear_grads()
        #return(self.w, self.b)
    
    def logos(self,sample):
        return(sigmoid(sample.dot(self.w) + self.b))

In [320]:
l1 = Layer(2, 1)

In [321]:
l1.w

array([[0.],
       [0.]], dtype=float32)

In [322]:
mu_0 = np.random.normal(loc=(-1, -1), scale=(1, 1))
mu_1 = np.random.normal(loc=(1, 1), scale=(1, 1))

In [323]:
mu_0

array([-1.25465211, -0.57366029])

In [324]:
mu_1

array([-0.56558939,  0.67500025])

In [339]:
t = np.array([0])

In [340]:
x = mu_0
x.shape

(2,)

In [341]:
for i in range(10000):
    y = l1(x)
    #if i % 100 == 0:
        #print(f'{log_loss(t, y)} + {diff_log_loss(t, y)}')
    d_y = diff_log_loss(t, y)
    l1.backward(d_y)
    l1.step()
print()

In [328]:
y

array([[0.0099028]])

In [329]:
log_loss(t, y)

array([0.00995216])

In [344]:
t1 = np.array([1])
x1 = mu_1

In [345]:
for i in range(10000):
    y = l1(x1)
    #if i % 100 == 0:
        #print(f'{log_loss(t1, y)} + {diff_log_loss(t1, y)}')
    d_y = diff_log_loss(t1, y)
    l1.backward(d_y)
    l1.step()

[0.10145966] + [[-0.17639991]]
[0.08190877] + [[-0.14378578]]
[0.07020206] + [[-0.12394964]]
[0.06225133] + [[-0.1103447]]
[0.05642265] + [[-0.10030197]]
[0.05192463] + [[-0.09251185]]
[0.04832327] + [[-0.08624932]]
[0.04535873] + [[-0.08107724]]
[0.04286509] + [[-0.07671483]]
[0.04073092] + [[-0.07297264]]
[0.03887828] + [[-0.06971761]]
[0.03725089] + [[-0.06685336]]
[0.03580699] + [[-0.06430816]]
[0.03451483] + [[-0.0620273]]
[0.0333498] + [[-0.05996832]]
[0.03229249] + [[-0.05809766]]
[0.03132752] + [[-0.05638862]]
[0.03044223] + [[-0.05481926]]
[0.02962635] + [[-0.05337172]]
[0.02887132] + [[-0.05203108]]
[0.02817] + [[-0.0507849]]
[0.02751635] + [[-0.04962263]]
[0.02690529] + [[-0.04853542]]
[0.0263324] + [[-0.04751551]]
[0.02579388] + [[-0.04655626]]
[0.02528647] + [[-0.04565195]]
[0.02480729] + [[-0.04479753]]
[0.02435387] + [[-0.04398866]]
[0.02392398] + [[-0.04322143]]
[0.02351565] + [[-0.04249238]]
[0.02312717] + [[-0.0417985]]
[0.02275699] + [[-0.04113704]]
[0.02240372] + [[